# Scotty HQ animation

reference

- https://gdal.org/programs/gdalbuildvrt.html



In [ ]:
! gdalbuildvrt --help

In [ ]:
! aws s3 ls dev-et-data/enduser/DelawareRiverBasin/drb150cAnnual/ | awk '{print $NF}' > filelist.txt

In [ ]:
fr=open('filelist.txt','r')
#print(fr.read())

In [ ]:
fl = fr.readlines()


In [ ]:
wl=[]
prefix = '/vsis3/dev-et-data/enduser/DelawareRiverBasin/drb150cAnnual/'
for fname in fl:
    print(fname.rstrip())
    prefix_name = prefix + fname.rstrip()
    wl.append(prefix_name)

In [ ]:
fw=open('vrtfilelist.txt','w')
for i in wl:
    print(i)
    fw.write(i)
    fw.write('\n')

In [ ]:
fw.close()

In [ ]:
! cat vrtfilelist.txt

In [ ]:
! gdalbuildvrt -resolution average -input_file_list vrtfilelist.txt myvrt.xml

In [ ]:
!ls

In [ ]:
import xarray as xr

In [ ]:
#help(xr.open_rasterio)

In [ ]:
wl[0]

In [ ]:
da = xr.open_rasterio(wl[0])

In [ ]:
da

# Read in an overview the ugly way

list the overviews

In [ ]:
import rasterio
src = rasterio.open(wl[0], 'r')
[src.overviews(i) for i in src.indexes]

In [ ]:
nsub = 4
nsub_half = int(nsub/2)
data = src.read(out_shape=(1, int(src.height / nsub), int(src.width / nsub)))

In [ ]:
src.crs

In [ ]:
src.shape

In [ ]:
src.is_tiled

In [ ]:
dar = xr.DataArray(data, dims=da.dims, coords=[da.band, da.y[nsub_half:-nsub_half:nsub], da.x[nsub_half:-nsub_half:nsub]])

dar.attrs = da.attrs

# 1/4 original size

In [ ]:
dar

In [ ]:
import hvplot.xarray
dar_masked = dar.where(dar<10000.0)
dar_masked = dar_masked.where(dar>0.0)
img_figure = dar_masked.hvplot.image(dynamic=False,rasterize=True,grid=True, invert=False, width=400, height=500, cmap='viridis')

In [ ]:
methods = dir(img_figure)
for meth in methods:
    if meth.startswith('his'):
        print(meth)

In [ ]:
img_figure

In [ ]:
img_figure.hist()

In [ ]:
da

In [ ]:
da_masked = da.where(da<10000.0)
da_masked = da_masked.where(da>0.0)
img_figure = da_masked.hvplot.image(dynamic=False,rasterize=True,grid=True, invert=False, width=400, height=500, cmap='viridis')


In [ ]:
img_figure

In [ ]:
da.shape

In [ ]:
siz1 = da.shape[1] * da.shape[2] * 150 * 8

In [ ]:
sizG = siz1 /1e9

In [ ]:
sizG

above is memory use estimate for the xarray for 150 years of data

In [ ]:
how_many=10

In [ ]:
def tif_to_da(tif):
    return xr.open_rasterio(tif)

In [ ]:
%%time
tifs = wl
dalist =[tif_to_da(tif) for tif in tifs[:how_many]]

da = xr.concat(dalist, dim='band')
da = da.rename({'band':'year'})

In [ ]:
da = da.assign_coords(year=range(1950,1950+how_many))

In [ ]:
ds = da.to_dataset(name='etasw')

# More on xarray DataSets and DataArrays later

In [ ]:
ds

### look ma - no attributes

In [ ]:
da

In [ ]:
da_masked = da.where(da<10000.0)
da_masked = da_masked.where(da>0.0)
img_figure = da_masked.hvplot.image(dynamic=False,rasterize=True,grid=True, invert=False, width=400, height=500, cmap='viridis')

In [ ]:
img_figure.hist()

# I think i'll create a full 150 year animation next!